In [ ]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import src.functions as src

fs=500
freq_bands={'delta' : [0,4],
            'theta' : [4,8],
            'alpha' : [8, 13],
            'beta' : [13,35],
            'high' : [35, 45],
            'all': [1.5,45],
            'low': [1.5,12]}

data2={}
labels2={}
path='Dataset2'
for files in os.listdir(path):
    folders=os.path.join(path,files)
    file2=[item for item in os.listdir(folders) if item.endswith('set')]
#--------------------------------------------------------------------------
    i=0
    label=[]
    while i!=4:
        for file_index, file_name in enumerate(file2):
            i=i+1
            path_ = os.path.join( folders, file_name )
            dados = mne.read_epochs_eeglab(path_)
            channel_names2 = dados.info['ch_names']
            subject_id = file_name[:3]
            
            # Extract labels
            try:
                Corr_labels = int(len(dados['150','160'].get_data()))
            except:
                Corr_labels=0
            try:
                Err_labels = int(len(dados['151','152','153','154','161','162','163','164'].get_data()))
            except:
                Err_labels=0
            label.append(np.array([0] * Corr_labels + [1] * Err_labels))
           
    #--------------------------------------------------------------------------    
            # Initialize channel data dictionary
            channel_data = {}
            data_channel=[]
            try:
                for epoch_values in dados['150','160'].get_data():
                    for channel_index, channel_values in enumerate(epoch_values):
                        channel_name = channel_names2[channel_index]
                        if channel_name not in channel_data:
                            channel_data[channel_name] = []
                        channel_data[channel_name].append(list(channel_values))
            except:
                pass
            try:
                for epoch_values in dados['151','152','153','154','161','162','163','164'].get_data():
                    for channel_index, channel_values in enumerate(epoch_values):
                        channel_name = channel_names2[channel_index]
                        if channel_name not in channel_data:
                            channel_data[channel_name] = []
                        channel_data[channel_name].append(list(channel_values))
            except:
                pass
            
            data2[file_name] = channel_data
        labels2[subject_id] = np.concatenate(label)

#--------------------------------------------------------------------------

rearranged_data = {}

# Iterate through each file in data2
for file_name, channel_data in data2.items():
    # Extract the subject ID from the file name
    subject_id = file_name.split('_')[0]
    
    # Initialize the subject in the rearranged_data dictionary if not already present
    if subject_id not in rearranged_data:
        rearranged_data[subject_id] = {}
    
    # Iterate through each channel in the current file
    for channel, data in channel_data.items():
        # Initialize the channel in the subject's dictionary if not already present
        if channel not in rearranged_data[subject_id]:
            rearranged_data[subject_id][channel] = []
        
        # Append the current file's data for the channel to the subject's channel data
        rearranged_data[subject_id][channel].append(data)

# Combine the data from different files of the same subject for each channel
for subject_id, channels in rearranged_data.items():
    for channel, data_list in channels.items():
        # Flatten the list of data arrays into a single list or concatenate them
        # Here we assume each data is a list or array; if not, adjust accordingly
        combined_data = []
        for data in data_list:
            combined_data.extend(data)
        rearranged_data[subject_id][channel] = combined_data

# Now rearranged_data is in the desired format
# You can print or further process the rearranged_data as needed

data2=rearranged_data
        

In [8]:
def data_pre_post(data):
    pre_response={}
    post_response={}
    for subject_id, dados in data.items():
        channel_data_pre={}
        channel_data_post={}
        for channel_name, values in dados.items():
            values_pre=[]
            values_post=[]
            for i in range(len(values)):
                
                index= int(len(values[i])/2)
                values_pre.append(values[i][0:index])
                values_post.append(values[i][index:-1])
            channel_data_pre[channel_name]= values_pre
            channel_data_post[channel_name]= values_post
            
        pre_response[subject_id] = channel_data_pre #(14,60,n_events,250)
        post_response[subject_id] = channel_data_post
    return pre_response, post_response

pre_response2, post_response2 = data_pre_post(data2)

In [9]:
f_pre2,S_pre2= src.getpsd(pre_response2,fs)    # S_pre ->(14,60,n_events,n_points)
f_post2,S_post2= src.getpsd(post_response2,fs)

In [10]:
pre_data_per_channel2= src.getdataperchannel(S_pre2) #(subject_id,60,n_events,n_points)
post_data_per_channel2= src.getdataperchannel(S_post2)

In [15]:
feature1= src.feature('fcz_features','.','.','theta','all',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2, f_post2)
feature2= src.feature('fcz_features','.','.','theta','high',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2)
feature3= src.feature('fcz_features','.','.','theta','all',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2)
feature4= src.feature('fcz_features','.','.','theta','high',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2)
feature5= src.feature('fcz_features','.','.','theta','other',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2)
feature6= src.feature('fcz_features','.','.','theta','other',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2)
feature7= src.feature('fcz_features','.','.','theta','theta',post_data_per_channel2,pre_data_per_channel2,freq_bands,f_post2,f_pre2)

In [64]:
data1= feature1
data2= feature2
data3= feature3
data4= feature4
data5= feature5
data6= feature6
data7= feature7

In [42]:
data8={}
data9={}
data10={}
data11={}
data12={}
data13={}
data14={}
data15={}
data16={}
for channel_name in post_data_per_channel2['P01'].keys():    
    feature8= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','all',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature9= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','high',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature10= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','all',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature11= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','high',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature12= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','theta',post_data_per_channel2,pre_data_per_channel2,freq_bands,f_post2,f_pre2))
    feature13= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','delta',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature14= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','alpha',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature15= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','delta',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature16= src.extend_list(src.feature('all_features',channel_name,channel_name,'theta','alpha',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    data8[channel_name]=feature8
    data9[channel_name]=feature9
    data10[channel_name]=feature10
    data11[channel_name]=feature11
    data12[channel_name]=feature12
    data13[channel_name]=feature13
    data14[channel_name]=feature14
    data15[channel_name]=feature15
    data16[channel_name]=feature16


In [43]:
data17={}
data18={}
for channel_name in post_data_per_channel2['P01'].keys():
    feature17= src.extend_list(src.feature('midfrontal_features',channel_name,channel_name,'theta','other',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature18= src.extend_list(src.feature('midfrontal_features',channel_name,channel_name,'theta','other',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    data17[channel_name]=feature17
    data18[channel_name]=feature18

In [51]:
data19={}
data20={}
data21={}
data22={}
data23={}
data24={}
data25={}
data26={}
data27={}
data28={}
data29={}
data30={}
for channel_name in post_data_per_channel2[subject_id].keys():        
    
    feature19= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','all',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature20= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','all',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature21= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','all',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature22= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','high',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature23= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','high',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature24= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','high',post_data_per_channel2,post_data_per_channel2,freq_bands,f_post2,f_post2))
    feature25= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','all',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature26= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','all',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature27= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','all',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature28= src.extend_list(src.feature('low_features',channel_name,channel_name,'delta','high',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature29= src.extend_list(src.feature('low_features',channel_name,channel_name,'alpha','high',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    feature30= src.extend_list(src.feature('low_features',channel_name,channel_name,'low','high',pre_data_per_channel2,pre_data_per_channel2,freq_bands,f_pre2,f_pre2))
    data19[channel_name]=feature19
    data20[channel_name]=feature20
    data21[channel_name]=feature21
    data22[channel_name]=feature22
    data23[channel_name]=feature23
    data24[channel_name]=feature24
    data25[channel_name]=feature25
    data26[channel_name]=feature26
    data27[channel_name]=feature27
    data28[channel_name]=feature28
    data29[channel_name]=feature29
    data30[channel_name]=feature30

In [52]:
feature31={}
feature32={}
FCZ_Cluster=['FZ','FC1','FCZ','FC2','CZ']
PE_Cluster=['CPZ','P1','PZ','P2','POZ']
for subject_id,values in post_response2.items():
    mean_all70_160_values = []
    mean_all200_500_values = []
    for i in range(len(next(iter(values.values())))):
        all70_160_values = []
        all200_500_values = []
        for channel_name, channel_values in values.items():   
            if channel_name in FCZ_Cluster:         
                all70_160_values.append(np.mean(channel_values[i][35:81]))
            elif channel_name in PE_Cluster:
                all200_500_values.append(np.mean(channel_values[i][100:-1]))
            else:
                continue

        mean_all70_160_values.append(np.mean(all70_160_values))
        mean_all200_500_values.append(np.mean(all200_500_values))
    feature31[subject_id]= mean_all70_160_values
    feature32[subject_id]= mean_all200_500_values

In [66]:
data31=src.extend_list(feature31)
data32=src.extend_list(feature32)

In [ ]:
import numpy as np

# Initialize the dictionaries to store the results
feature33 = {}
feature34 = {}

# Iterate over each subject in the pre_response2 dictionary
for subject_id, values in pre_response2.items():
    # Iterate over each channel in the subject's data
    for channel_name, channel_values in values.items():
        # If the channel is not yet in the feature dictionaries, initialize it
        if channel_name not in feature33:
            feature33[channel_name] = []
        if channel_name not in feature34:
            feature34[channel_name] = []
        
        # Iterate over each event in the channel values
        for i in range(len(channel_values)):
            # Calculate the mean values for the specified ranges
            mean_500_250 = np.mean(channel_values[i][0:126])
            mean_250_0 = np.mean(channel_values[i][125:-1])
            
            # Append the mean values to the corresponding channel's list in the feature dictionaries
            feature33[channel_name].append(mean_500_250)
            feature34[channel_name].append(mean_250_0)


In [83]:
data33=feature33
data34=feature34

In [84]:

# Initialize the dictionaries to store the results
feature35 = {}
feature36 = {}

# Iterate over each subject in the pre_response2 dictionary
for subject_id, values in post_response2.items():
    # Iterate over each channel in the subject's data
    for channel_name, channel_values in values.items():
        # If the channel is not yet in the feature dictionaries, initialize it
        if channel_name not in feature35:
            feature35[channel_name] = []
        if channel_name not in feature36:
            feature36[channel_name] = []
        
        # Iterate over each event in the channel values
        for i in range(len(channel_values)):
            # Calculate the mean values for the specified ranges
            mean_0_250 = np.mean(channel_values[i][0:126])
            mean_250_500 = np.mean(channel_values[i][125:-1])
            
            # Append the mean values to the corresponding channel's list in the feature dictionaries
            feature35[channel_name].append(mean_0_250)
            feature36[channel_name].append(mean_250_500)


In [85]:
data35=feature35
data36=feature36

In [ ]:
import pandas as pd

dicts=[]
for feature_index in range(1,37):
    feature_corr_key = 'data{}'.format(feature_index)
    dicts.append(globals()[feature_corr_key])



combined_df = pd.DataFrame()
# Iterate through each dictionary and its index
for index, feature_dict in enumerate(dicts, start=1):
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(feature_dict)
    # Rename the columns to include the iteration number
    df.columns = [f'{index}_{col}' for col in df.columns]
    # Concatenate the DataFrame to the combined DataFrame
    combined_df = pd.concat([combined_df, df], axis=1)

print(combined_df)

In [ ]:
combined_df['labels'] = src.extend_list(labels2)
print(combined_df)

In [90]:
combined_df.to_csv('dataset2_features.csv', index=False)